In [1]:
# Hospital
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
#from api_keys import g_key

#Matt's code
import sys
path = "/Users/meria/Desktop"
sys.path.insert(0, path)
from apifile import API_KEY
gkey= API_KEY["api_google"]

In [2]:
# Read the  data lat and lon from cities
cities = pd.read_csv("../data/GeoIDs - City.csv")
cities.head()
#Create a new column to storage the Park Score for each city
#Note: The Park Score of each city is defined as:
#      average of the ratings available on google api  for places target_type = "park"
#      around 5000 meters of each city location

cities["Park Score"] = ""
cities.head()

,cityid,cityname,stateabbrev,statename,statefips,lat,lon,city_pop2019,Park Score
0,1,Los Angeles,CA,California,6,34.05,-118.24,10039107,
1,2,New York City,NY,New York,36,40.71,-74.01,8336817,
2,3,Chicago,IL,Illinois,17,41.88,-87.63,5150233,
3,4,Houston,TX,Texas,48,29.76,-95.37,4713325,
4,5,Phoenix,AZ,Arizona,4,33.45,-112.07,4485414,


In [3]:
#Gmaps
gmaps.configure(api_key = gkey)
#location
location = cities[["lat","lon"]]

In [4]:
#geocordinates
target_radius = 5000 #meters
target_type = "park"
ratings = "rating"

#set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "rating": ratings,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [5]:
#Empty counter to iterate over the cities in the for loop
k=0

for col,row in cities[:].iterrows():
    #Get  the Latitude and Longitude.
    lat =row["lat"]
    lng = row["lon"]  
    params["location"] = f"{lat},{lng}"

    # Request to get the Park Rating using url and parameters.
    response = requests.get(base_url, params=params)
    city = response.json()
    
    # empty list to get the rating values around the city in the specific radius 
    rating = []
    no_rating = 0
    
    #looking for park rating and storing the values in a list when a value is available
    for i in range (0,len(city["results"])+1):
        try:
            rating.append(city["results"][i]["rating"])

        except:
            no_rating = no_rating +1
    
    #print(rating)
    
    #calculate the average of the rating values found
    avg_rating = sum(rating)/len(rating)
    #print(avg_rating)
    
    #insert the rating average in an specific location (row = k) at the cities Data Frame, Health score column
    cities["Park Score"].iloc[(k)]=avg_rating
    #pprint(city)
    #counter that will go to the next city in the DataFrame
    k=k+1

C:\Users\meria\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [9]:
cities.head()

,cityid,cityname,stateabbrev,statename,statefips,lat,lon,city_pop2019,Park Score,Park Rank
0,1,Los Angeles,CA,California,6,34.05,-118.24,10039107,4.34737,24.0
1,2,New York City,NY,New York,36,40.71,-74.01,8336817,4.585,49.0
2,3,Chicago,IL,Illinois,17,41.88,-87.63,5150233,4.67647,52.0
3,4,Houston,TX,Texas,48,29.76,-95.37,4713325,4.52941,46.0
4,5,Phoenix,AZ,Arizona,4,33.45,-112.07,4485414,4.425,31.0


In [7]:
# ranked 1 (worst) to 5(best)
cities["Park Rank"]= cities["Park Score"].rank(ascending=True)


In [11]:
#Exporting the DataFrame with the rank and scores in a csv file and reading the DataFrame
ParkData_df = cities[["cityid","Park Score"]]  

ParkData_df.to_csv("../output_files/parks_score.csv")
ParkData_df


,cityid,Park Score
0,1,4.34737
1,2,4.585
2,3,4.67647
3,4,4.52941
4,5,4.425
5,6,4.45789
6,7,4.5
7,8,4.35333
8,9,4.10667
9,10,4.44444
